# 读取CSV文件

In [1]:
import pandas as pd
import jieba
import numpy as np
from collections import Counter

In [2]:
content=pd.read_csv(r'/Users/yang/Downloads/datasource-master/sqlResult_1558435.csv',encoding='gb18030')
content.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [3]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
id         89611 non-null int64
author     79396 non-null object
source     89609 non-null object
content    87054 non-null object
feature    89611 non-null object
title      89577 non-null object
url        87144 non-null object
dtypes: int64(1), object(6)
memory usage: 4.8+ MB


In [4]:
content[content['content']=='?\r\n'].head()

,id,author,source,content,feature,title,url
2310,87307,廖越 PF062,凤凰财经,?\r\n,"{""type"":""财经"",""site"":""凤凰"",""commentNum"":""0"",""joi...",一张图看懂一带一路的钱都从哪来？,http://finance.ifeng.com/a/20170515/15374639_0...
5039,84578,张雪2 PF032,每日涨停板复盘,?\r\n,"{""type"":""财经"",""site"":""凤凰"",""commentNum"":""1"",""joi...",复盘：创小板继续反弹 特斯拉站风口,http://finance.ifeng.com/a/20170620/15470267_0...


In [5]:
content.drop([2310,5039],inplace=True)
content[content['content']=='?\r\n'].head()

,id,author,source,content,feature,title,url


In [6]:
content2=content.dropna(subset=['content'])

In [7]:
content2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87052 entries, 0 to 89610
Data columns (total 7 columns):
id         87052 non-null int64
author     76923 non-null object
source     87050 non-null object
content    87052 non-null object
feature    87052 non-null object
title      87052 non-null object
url        87052 non-null object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


# 数据预处理

In [8]:
corpus=[' '.join(list(jieba.cut(str(content2.iloc[i].content)))) for i in range(60000)]
source=[content2.iloc[i].source for i in range(60000)]
target=[1 if i=='新华社' or i=='新华网' else 0 for i in source]
y=np.array(target)
Counter(y)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/kl/7_vv7v_15wnf5q65wh8tl5jh0000gn/T/jieba.cache
Loading model cost 1.044 seconds.
Prefix dict has been built succesfully.


Counter({0: 8219, 1: 51781})

#### sklearn.feature_extraction.text  之 CountVectorizer用法 示例

In [9]:
#from sklearn.feature_extraction.text import CountVectorizer
#corpus1 = [
#...     'This is the first document.',
#...     'This document is the second document.',
#...     'Is this the first document?',]
#vectorizer = CountVectorizer()
#X1 = vectorizer.fit_transform(corpus1)
#print(vectorizer.get_feature_names())
#print(X1.toarray())  

#### 生成特征向量数据集（计算每个词的TF-IDF）

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
X=vectorizer.fit_transform(corpus)
np.shape(X)

(60000, 215410)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer=TfidfTransformer()
tfidf=transformer.fit_transform(X)
tfidf.toarray

<bound method _cs_matrix.toarray of <60000x215410 sparse matrix of type '<class 'numpy.float64'>'
	with 5648612 stored elements in Compressed Sparse Row format>>

# KNN分类器

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=3,weights='uniform')
knn.fit(tfidf,y)
y_pred=knn.predict(tfidf)

In [14]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_pred, y, average='macro')  

(0.9221275600745913, 0.764966363467507, 0.8082301536216807, None)

### 手动调参

（1）(n_neighbors=3,weights='uniform')，precision_recall_f1: (0.9221275600745913, 0.764966363467507, 0.8082301536216807)

（2）(n_neighbors=4,weights='uniform')，precision_recall_f1: (0.8895241379310345, 0.8891666666666667, 0.8585793380035288)

（3）(n_neighbors=5,weights='uniform')，precision_recall_f1: (0.8877569546357776, 0.8815833333333334, 0.8421000803638186)

（4）(n_neighbors=3,weights='distance'),precision_recall_f1: (0.9008032565275556, 0.8958333333333334, 0.8687527015061292)

（5）(n_neighbors=4,weights='distance'),precision_recall_f1: (0.9007190859477344, 0.89575, 0.8686106220331913)

（6）(n_neighbors=5,weights='distance'),precision_recall_f1 :(0.8989237818730857, 0.89175, 0.8609381836951601)

所以，最优暂定(n_neighbors=3,weights='distance')。

### 自动调参过程---自己的电脑，太慢了，只列代码。

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier
#knn_clf = KNeighborsClassifier()

#from sklearn.model_selection import GridSearchCV
#param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]
#grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3, n_jobs=-1)
#grid_search.fit(Xtrain, ytrain)


In [304]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y,y_pred)

array([[ 8057,   162],
       [ 7044, 44737]])

### 使用NearestNeighbors找到每篇文章的最相近文章的索引、二者距离,并构建DataFrame。

In [15]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(tfidf)
distances, indices = neigh.kneighbors(tfidf)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/neighbors/base.py:216: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [16]:
import pandas as pd
content_df=pd.DataFrame(distances[:,1],columns=['distance'])
content_df['origin_index']=indices[:,0]
content_df['similar_artical_index']=indices[:,1]

In [17]:
content_df.head()

,distance,origin_index,similar_artical_index
0,1.000000,0,5336
1,1.000000,1,5336
2,1.000000,2,5336
3,0.000000,3,39
4,0.235425,4,5489


In [18]:
content_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
distance                 60000 non-null float64
origin_index             60000 non-null int64
similar_artical_index    60000 non-null int64
dtypes: float64(1), int64(2)
memory usage: 1.4 MB


### 选出疑似抄袭文章

In [22]:
def get_cx_index(y,y_pred):
    cx_index=[]
    a=np.arange(len(y))
    b=a[y==0]
    c=a[y_pred==1]
   
    for i in b:
        if i in c:
            cx_index.append(i)
    return cx_index

cx_index=get_cx_index(y,y_pred)
content_cx=content_df.iloc[cx_index]
print('疑似抄袭文章篇数',len(cx_index))

疑似抄袭文章篇数 162


In [25]:
print('下表为抄袭文章索引和疑被抄袭文章索引，以及相似度信息，distance越小，越相似')
content_cx.head()

下表为抄袭文章索引和疑被抄袭文章索引，以及相似度信息，distance越小，越相似


,distance,origin_index,similar_artical_index
55,0.919737,55,54552
63,0.873234,63,49048
97,0.920622,97,15291
113,0.986465,113,52861
151,0.744315,151,36960


##### 先选取distance小于0.5的进行抽样查看，发现大多是原文引用。增加了“据新华社电……”等字样

In [26]:
content_cx[(content_cx['distance'])<0.5]

,distance,origin_index,similar_artical_index
1071,0.123810,1071,1063
1101,0.000000,1101,1190
1607,0.210366,1607,3950
3067,0.000000,3047,3067
3819,0.451402,3819,37913
4603,0.000000,3197,4603
5076,0.406452,5076,5133
6124,0.217447,6124,5304
6444,0.327121,6444,555
8243,0.489581,8243,8075


In [27]:
print('疑似抄袭文章：')
content2.iloc[1607]['content']

疑似抄袭文章：


'（原标题：美军找到数具撞船事故失踪者遗体）\r\n据新华社电?美国海军18日发表声明称，在美军“菲茨杰拉德”号驱逐舰与菲律宾籍货船相撞事故中失踪的美军人员已经死亡，死者遗体在美军驱逐舰进水的住舱内找到。\r\n17日凌晨2时许，美国海军第七舰队“菲茨杰拉德”号驱逐舰在返回驻地神奈川县横须贺港途中，在横须贺市西南56海里水域与一艘挂有菲律宾国旗的货船相撞。“菲茨杰拉德”号舰体右侧吃水线上下受损，7名美军人员下落不明。\r\n事故发生后，美国海军拖船前往事发水域，把“菲茨杰拉德”号驱逐舰拉回基地。美国海军第七舰队18日发表声明称，失踪人员遗体已经找到。“搜救人员进入撞击中受损区域，在进水的住舱内找到失踪人员。”\r\n美联社报道，撞击致两处住舱、一处机房和无线电室进水，船体右侧变形，吃水线以下受损情况尚有待明确。事故发生时，“菲茨杰拉德”号上有美军超过200人，大多数人已睡着。\r\n美联社援引美军驻横须贺海军发言人的话报道，7名失踪人员并未全部找到。该发言人没有说明已经找到的确切人数。\r\n海军第七舰队声明写道：“家属已经得到通知。在这一困难时刻，海军将向他们提供一切必要支持。”\r\n死者姓名将在晚些时候公布。船只相撞原因仍在调查。\r\n第七舰队司令约瑟夫·奥库安在18日晚些时候举行的新闻发布会上说，“菲茨杰拉德”号被撞后，受损严重，吃水线下被撞出一条大口子。“菲茨杰拉德”眼看着就要沉没，幸而船员反应迅速，竭力抢救。\r\n奥库安说，海上搜救行动已经叫停。“菲茨杰拉德”号修理完毕后将继续服役，修理工作耗时预计“不会超过一年”。\r\n美国海军“菲茨杰拉德”号驱逐舰全长154米，满载排水量为8000吨。与其相撞的菲律宾籍货船全长222.6米，满载排水量为2.9万吨。日本海上保安厅17日接到货船紧急求救电话，称2时20分左右发生撞击。\r\n撞击事故造成多名美军人员受伤，包括“菲茨杰拉德”号指挥官布赖斯·本森。菲律宾籍货船上有20名船员，无人受伤。货船船体可以看到凹痕和划痕，但没有大伤。\r\n'

In [28]:
print('新华社文章：')
content2.iloc[3950]['content']

新华社文章：


'美国海军１８日发表声明称，在美军“菲茨杰拉德”号驱逐舰与菲律宾籍货船相撞事故中失踪的美军人员已经死亡，死者遗体在美军驱逐舰进水的住舱内找到。\r\n１７日凌晨２时许，美国海军第七舰队“菲茨杰拉德”号驱逐舰在返回驻地神奈川县横须贺港途中，在横须贺市西南５６海里水域与一艘挂有菲律宾国旗的货船相撞。“菲茨杰拉德”号舰体右侧吃水线上下受损，７名美军人员下落不明。\r\n事故发生后，美国海军拖船前往事发水域，把“菲茨杰拉德”号驱逐舰拉回基地。美国海军第七舰队１８日发表声明称，失踪人员遗体已经找到。“搜救人员进入撞击中受损区域，在进水的住舱内找到失踪人员。”\r\n美联社报道，撞击致两处住舱、一处机房和无线电室进水，船体右侧变形，吃水线以下受损情况尚有待明确。事故发生时，“菲茨杰拉德”号上有美军超过２００人，大多数人已睡着。\r\n失踪人员遗体已运往海军医院，以确认身份。声明没有说明找到多少名死者，也没有说明是否还有人员下落未明。\r\n美联社援引美军驻横须贺海军发言人的话报道，７名失踪人员并未全部找到。该发言人没有说明已经找到的确切人数。\r\n海军第七舰队声明写道：“家属已经得到通知。在这一困难时刻，海军将向他们提供一切必要支持。”\r\n死者姓名将在晚些时候公布。船只相撞原因仍在调查。\r\n第七舰队司令约瑟夫·奥库安在１８日晚些时候举行的新闻发布会上说，“菲茨杰拉德”号被撞后，受损严重，吃水线下被撞出一条大口子。“菲茨杰拉德”眼看着就要沉没，幸而船员反应迅速，竭力抢救。\r\n奥库安说，海上搜救行动已经叫停。“菲茨杰拉德”号修理完毕后将继续服役，修理工作耗时预计“不会超过一年”。\r\n美国海军“菲茨杰拉德”号驱逐舰全长１５４米，满载排水量为８０００吨。与其相撞的菲律宾籍货船全长２２２．６米，满载排水量为２．９万吨。日本海上保安厅１７日接到货船紧急求救电话，称２时２０分左右发生撞击。\r\n撞击事故造成多名美军人员受伤，包括“菲茨杰拉德”号指挥官布赖斯·本森。菲律宾籍货船上有２０名船员，无人受伤。货船船体可以看到凹痕和划痕，但没有大伤。\r\n'

##### 选取0.5<distance<0.7的，发现文章主题相同，有大量重复信息和相同句子。接下来，基于此类文章进行抄袭句子定位。

In [29]:
content_cx[(content_cx['distance']<0.7)  & (content_cx['distance']>0.5)]

,distance,origin_index,similar_artical_index
165,0.593684,165,15009
687,0.599947,687,54608
1424,0.547903,1424,21303
1627,0.647424,1627,20489
2424,0.534906,2424,19211
2891,0.673483,2891,22428
3088,0.673964,3088,5133
3206,0.612401,3206,50010
3482,0.504431,3482,37913
3710,0.563636,3710,4496


In [30]:
print('疑似抄袭文章：')
content2.iloc[165]['content']

疑似抄袭文章：


'\u3000\u3000“由于东航物流本身股本高达41亿，规定的单个员工持股上限1%，实际上金额有4100万之多。对目前125名持股的员工而言，都是要拿出真金白银来的，持了股，就相当于背上身家在打拼，必须奋勇直前。”东航物流总经理李九鹏在接受《经济参考报》记者采访时表示。\r\n\u3000\u3000作为公司的管理层，李九鹏同时也是公司员工持股的一名代表，并通过无限责任的质押贷款筹得资金3000多万持股。用他自己的话，公司如果干不好，不仅自己工作前途没有了，自己可能下半辈子都需要背负这笔巨额债务还债。换句话说，公司的前途跟员工个人已经牢牢绑在一起。\r\n\u3000\u3000作为混改确定的第一批试点单位，东航集团旗下的东航物流开始“试水”改革，除了放弃绝对控制权外，同时率先落地员工持股系列改革。记者从东航获悉，在引入四家投资方后，目前新的东航物流计划员工持股的比例高达10%，首批持股人员主要为公司的中高级管理人员及核心员工，目前已有125名员工持股，涉及出资金额近4亿元。首批员工持股总额为东航物流本次增资扩股后注册资本的8%，预留的2%则仍由员工持股平台管理。\r\n\u3000\u3000作为混改的重要组成部分，员工持股也被称为本轮改革中最难啃的“硬骨头”。一位地方国企人士坦言，员工持股改革就是要建立收益共享、风险共担的长效激励约束机制，保持核心人才队伍的稳定性，优化股权结构，完善公司治理机制。相较于其他形式的激励，员工持股方式能更有效地把国有资本和人力资本及创造性劳动结合起来，发挥国有资本作用的同时，激发人力资源的创造性劳动。同时，改变以往“大锅饭”、“论资排辈”的国企特点，对关键岗位和优秀人才启动员工持股计划，也有助于优化收入分配制度，可以有效监督企业的规范运作。\r\n\u3000\u3000不容忽视的是，虽然国企混改中的员工持股试点正在加速，但是目前的不确定因素仍比较多。记者了解到，比如在持股比例方面，相当多的国企偏于谨慎，员工持股的比例较低，这难以显著激发广大员工积极性，并与企业形成利益共同体。另外，在股票来源上，多数国企的员工持股计划完全靠定向增发，结构比较单一。不仅如此，在参与试点的员工范围上，不少国企明确为“骨干员工”，但哪些是“骨干员工”目前没有清晰界定。\r\n\u3000\u3000“员工持股改革问题上曾经走过弯路，这也给我们的改革提

In [31]:
print('新华社文章：')
content2.iloc[15009]['content']

新华社文章：


'\u3000\u3000新华社北京４月５日新媒体专电（记者杨烨）《经济参考报》记者日前从多个省份获悉，目前包括北京、上海、陕西、湖南、湖北、四川、山东、重庆等近２０个省市明确要推进地方国企员工持股试点，多个省份正在研究制定混合所有制企业开展员工持股试点的落实意见或方案，设定改革时间表。启动试点的同时，试点企业名单将进一步扩围。\\n\u3000\u3000作为混合所有制改革的重要组成部分，员工持股无疑是国企改革中最具挑战性和敏感性的难题之一。按照国务院国资委计划，下一步将在主业处于充分竞争行业和领域的商业类企业中，分别从中央企业选择１０家，地方国有企业选择５至１０家开展首批试点；２０１６年启动实施首批试点，成熟一户开展一户，２０１８年年底进行阶段性总结，视情况适时扩大试点。\\n\u3000\u3000“地方版落地方案实际上就是在总体方案的基础上，各个地方根据各地的产业结构、企业特点等来制定相应的细化方案，为的就是加快改革落地，不少省份都明确提出，要扩大试点企业，一些省份的试点企业名单也都在甄选上报。”一位地方国资人士告诉《经济参考报》记者。\\n\u3000\u3000以四川为例，记者从四川省国资委获悉，四川省已经制定并即将出台混合所有制企业员工持股指导意见，并设定改革“时间表”。下一步将在全省范围内选择５至１０户企业（其中，省属企业３至５户，市、州属以下企业３至５户）开展试点工作。按照试点意见，将优先支持人才资本和技术要素贡献占比较高的转制科研院所、高新技术企业、科技服务型企业开展员工持股试点。持股员工将优先设定在关键岗位工作，并对公司经营业绩和持续发展有直接或较大影响的科研人员、经营管理人员和业务骨干。\\n\u3000\u3000记者同时了解到，目前重庆市国资委也正在积极制定国有企业开展员工持股试点的实施办法，在划定７家试点企业的基础上指导企业积极推进改革。\\n\u3000\u3000北京市的员工持股试点工作将集中在新能源、生活性服务业、文化创意业等领域。目前，北京市已经初步圈定北汽新能源、庆丰包子铺、正达坤顺等６家试点企业，均为竞争类企业。\\n\u3000\u3000一位地方国企人士坦言，员工持股改革就是要建立收益共享、风险共担的长效激励约束机制，保持核心人才队伍的稳定性，优化股权结构，完善公司治理机制。相较于其他形式的激\\n励，员工持股方式能更有

##### 基于以上两篇文章，使用编辑距离editdistance，定位抄袭句子。

##### 编辑距离：又称Levenshtein距离，是指两个字串之间，由一个转成另一个所需的最少编辑操作次数，如果它们的距离越大，说明它们越是不同。许可的编辑操作包括将一个字符替换成另一个字符，插入一个字符，删除一个字符。

In [32]:
import editdistance

def find_similar_sentence(candidate,raw):
    similist = []
    cl = candidate.strip().split('。')
    ra = raw.strip().split('。')
    for c in cl:
        for r in ra:
            similist.append([c,r,editdistance.eval(c,r)])
    sort=sorted(similist,key=lambda x:x[2])
    for c,r,ed in sort:
        if c!='' and r!='' and ed<10:
            print('怀疑抄袭句:{0}\n相似原句:{1}\neditdistance:{2}\n'.format(c,r,ed))

            
candidate=content2.iloc[165]['content']
raw=content2.iloc[15009]['content']     

find_similar_sentence(candidate,raw)


怀疑抄袭句:同时，改变以往“大锅饭”、“论资排辈”的国企特点，对关键岗位和优秀人才启动员工持股计划，也有助于优化收入分配制度，可以有效监督企业的规范运作
相似原句:同时，改变以往“大锅饭”、“论资排辈”的国企特点，对关键岗位和优秀人才启动员工持股计划，也有助于优化收入分配制度，可以有效监督企业的规范运作
editdistance:0

怀疑抄袭句:另外，在股票来源上，多数国企的员工持股计划完全靠定向增发，结构比较单一
相似原句:另外，在股票来源上，多数国企的员工持股计划完全靠定向增发，结构比较单一
editdistance:0

怀疑抄袭句:不仅如此，在参与试点的员工范围上，不少国企明确为“骨干员工”，但哪些是“骨干员工”目前没有清晰界定
相似原句:不仅如此，在参与试点的员工范围上，不少国企明确为“骨干员工”，但哪些是“骨干员工”目前没有清晰界定
editdistance:0

怀疑抄袭句:与此同时，员工持股需要注意底线，不能搞全民持股，员工持股应设立负面清单，明确哪些能做哪些不能做，比如增量优先，科技企业优先，不同类型的企业，员工持股的比例也需划分
相似原句:与此同时，员工持股需要注意底线，不能搞全民持股，员工持股应设立负面清单，明确哪些能做哪些不能做，比如增量优先，科技企业优先，不同类型的企业，员工持股的比例也需划分
editdistance:0

怀疑抄袭句:此外，持股也要程序化、公开化
相似原句:此外，持股也要程序化、公开化
editdistance:0

怀疑抄袭句:相较于其他形式的激励，员工持股方式能更有效地把国有资本和人力资本及创造性劳动结合起来，发挥国有资本作用的同时，激发人力资源的创造性劳动
相似原句:相较于其他形式的激\n励，员工持股方式能更有效地把国有资本和人力资本及创造性劳动结合起来，发挥国有资本作用的同时，激发人力资源的创造性劳动
editdistance:2

怀疑抄袭句:
　　不容忽视的是，虽然国企混改中的员工持股试点正在加速，但是目前的不确定因素仍比较多
相似原句:\n　　不容忽视的是，虽然国企混改中的员工持股试点正在加速，但是目前的不确定因素仍比较多
editdistance:2

怀疑抄袭句:一位地方国企人士坦言，员工持股改革就是要建立收益共享、风险共担的长效激励约束机制，保持核心人才队伍的稳定性，优化股权结构，完善公司治理机制
